In [1]:
import numpy as np
from numpy.typing import NDArray
from pathlib import Path
from pprint import pprint

from tarea2.algorithms import *
from tarea2.io import *
from tarea2.gen import *


# Obtener los datos

In [5]:
# DEFINIR PARÁMETROS DE SIMPLEX
###############################

def ayudantia11():
  A = np.array([
    [1, -2, 1, 0],
    [-1, 1, 0, 1]
  ], dtype=np.float64)

  b = np.array([
    [4],
    [3]
  ])

  c = np.array([
    [-1],
    [-3],
    [0],
    [0],
  ])

  B = np.array([2, 3])
  NB = np.array([0, 1])
  return A, b, c, B, NB


def guia5_p2():
  A = np.array([
    #0  1  2  3  4  5
    [1, 2, 3, 1, 1, 0],
    [1, 1, 2, 3, 0, 1],
  ], dtype=np.float64)

  b = np.array([
    [5],
    [3]
  ])

  c = np.array([
    [-2],
    [-3],
    [-5],
    [-4],
    [0],
    [0],
  ])

  B = np.array([1, 2])
  NB = np.array([0, 3, 4, 5])

  return A, b, c, B, NB


def guia5_p3():
  A = np.array([
    #0  1  2  3  4  5
    [2, 3, 1, 0],
    [-1, 1, 0, 1],
  ], dtype=np.float64)

  b = np.array([
    [6],
    [1]
  ])

  c = np.array([
    [-1],
    [-3],
    [0],
    [0]
  ])

  B = np.array([2, 3])
  NB = np.array([0, 1])
  return A, b, c, B, NB


def youtube1_bland():
  # https://www.youtube.com/watch?v=tYprNf6JrdY
  A = np.array([
    #0  1  2  3  4  5
    [1,  1, 1, 0, 0],
    [2, -1, 0, 1, 0],
    [1,  0, 0, 0, 1],
  ], dtype=np.float64)

  b = np.array([
    [6],
    [0],
    [2],
  ])

  c = np.array([
    [3],
    [2],
    [0],
    [0],
    [0],
  ])

  B = np.array([2, 3, 4])
  NB = np.array([0, 1])
  return A, b, c, B, NB


def documento_cartagena99():
  # https://www.cartagena99.com/recursos/alumnos/apuntes/Ciclo_simplex.pdf
  A = np.array([
    #0  1  2  3      4   5  6
    [1, 0, 0, 0.25, -8, -1,   9],
    [0, 1, 0, 0.5, -12, -0.5, 3],
    [0, 0, 1, 0,     0,  1,   0],
  ], dtype=np.float64)

  b = np.array([
    [0],
    [0],
    [1],
  ])

  c = np.array([
    [0],
    [0],
    [0],
    [-0.75],
    [20],
    [-0.5],
    [6],
  ])

  B = np.array([0, 1, 2])
  NB = np.array([3, 4, 5, 6])
  return A, b, c, B, NB


# OBTENER PARÁMETROS Y VARIABLES
################################

A, b, c, B, NB = guia5_p2()

# COMIENZA ALGORITMO SIMPLEX
############################
# ENTRADAS: A, b, c, B, NB

def simplex(A, b, c, B, NB, DEBUG=False):
  X = np.zeros(shape=c.shape)

  for i in range(10):
    if DEBUG:
      print("A:")
      pprint(A)

      print("b:")
      pprint(b)

      print("c:")
      pprint(c)

      print("B:")
      pprint(B)

      print("A[B]:")
      pprint(A[:,B])

      print("NB:")
      pprint(NB)

      print("A[NB]:")
      pprint(A[:,NB])


    X[B] = np.dot(np.linalg.inv(A[:,B]), b)
    X[NB] = 0

    if DEBUG:
      print("X:")
      pprint(X)

      print("X[B]:")
      pprint(X[B])

      print("X[NB]:")
      pprint(X[NB])

    print("\nX es solución:")
    if (X >= 0).all():
      print("- básica factible ")
    else:
      print("- no es solución básica factible. Termino de Simplex")
      return np.zeros(shape=X.shape)

    if (X[B] == 0).any():
      print("- degenerada")
      pprint(lista_direccion_costos_reducidos(X, A, c, B, NB))
      return X


    X_cr = costos_reducidos(A, c, B, NB, DEBUG)


    if (X_cr >= 0).all():
      print(f"- óptima de valor {X * c}")

      if (X_cr == 0).any():
        print("- Hay múltiples óptimos")
      else:
        print("- Es óptimo único")

      return X

    else:
      print("- no es óptima")


    # DETERMINAR DIRECCIONES Y COSTOS REDUCIDOS
    ###########################################

    # Lo más sensato es empezar con una solución básica factible
    # Luego derivar la base, no-base y seguir.
    d_cr_list = lista_direccion_costos_reducidos(X, A, c, B, NB)

    if DEBUG:
      print("\nLista de direcciones y costos reducidos: ")
      pprint(d_cr_list)


    # ENCONTRAR DIRECCIÓN Y PASO
    ############################

    if DEBUG:
      print("\nCosto básico solución básica X_B: ")
      pprint(c[B])

    # Determinar dirección con menor costo reducido
    # (NB_i, d, cr)
    d_cr_min = min(d_cr_list, key=lambda x: x[2])

    if DEBUG:
      print("\nDirección de menor costo reducido: ")
      pprint(d_cr_min)

    # Obtener componentes de la dirección de menor costo reducido que:
    # - Pertenezcan a la base.
    # - Sean menores a cero.

    B_menores_a_cero = [B[i] for i, d_i in enumerate( d_cr_min[1][B] ) if d_i < 0]

    if DEBUG:
      print(f"{B_menores_a_cero = }")

    if len(B_menores_a_cero) == 0:
      print("El salto se hace infinito")
      print("El costo óptimo está indeterminado y se termina el algoritmo")
      break

    # Obtener la lista de coeficientes de desplazamientos posibles
    phi_list = [(- X[i] / d_cr_min[1][i], i) for i in B_menores_a_cero]

    # Quedarse con el menor, y determinar su índice (El que sale)
    phi, B_sale = min(phi_list)

    if DEBUG:
      print(f"{phi_list = }")
      print(f"{phi = }")


    # ACTUALIZAR BASE Y NO_BASE
    ########################################

    NB_entra = d_cr_min[0]
    B_sale_row =   np.where(B == B_sale)[0][0]
    NB_entra_row = np.where(NB == NB_entra)[0][0]

    if DEBUG:
      print()
      print(f"Entra a la base: {NB_entra}")
      print(f"Sale de la base: {B_sale}")

    B.put(B_sale_row, NB_entra)
    NB.put(NB_entra_row, B_sale)

    if DEBUG:
      print()
      print("B actualizado:")
      pprint(B)
      print("NB actualizado:")
      pprint(NB)

      print()
      print()
      print()
      print()
  return X


X = simplex(A, b, c, B, NB, DEBUG=False)
print("La solución encontrada es:")
pprint(X)


X es solución:
- básica factible 
- óptima de valor [[-0.]
 [-3.]
 [-5.]
 [-0.]
 [ 0.]
 [ 0.]]
- Hay múltiples óptimos
La solución encontrada es:
array([[0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.]])
